In [77]:
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import pandas as pd
from datetime import datetime
import numpy as np

df = pd.read_csv('stations_data.csv', sep=',')
def map_to_five(x):
    return min(x, 5)
df['rounded_last_update_5mins'] = pd.to_datetime(df['rounded_last_update_5mins'])
df['time_as_fraction'] = df['rounded_last_update_5mins'].dt.hour * 3600 + df['rounded_last_update_5mins'].dt.minute * 60 + df['rounded_last_update_5mins'].dt.second
weekday=df['rounded_last_update_5mins'].dt.dayofweek
weekdaydf = pd.get_dummies(weekday, prefix='day_of_week')
df = pd.concat([df, weekdaydf], axis=1)



In [78]:

# Step 2: Convert these seconds to a fraction of the day
df['time_as_fraction'] = df['time_as_fraction'] / 86400
# Applying the function to the 'available_bike_stands' column
#y_set = pd.DataFrame(df['available_bike_stands'].apply(map_to_five))
y_set = df['available_bike_stands']
y_set =  pd.DataFrame(y_set)
# Ensure x_set is in the correct shape
x_set = df[['number','time_as_fraction','day_of_week_0','day_of_week_1','day_of_week_2','day_of_week_3','day_of_week_4','day_of_week_5','day_of_week_6','temp','feels_like']]
#x_set =df.select_dtypes(include=['number'])
print(df.columns)
#x_set = df[['number','time_as_fraction','temp','feels_like']]
#x_set = df.drop(['available_bike_stands',"name","weather_desc","weather_brief",], axis=1)



# For the remainder dataset, separate X and Y as you did initially


Index(['number', 'name', 'position_lat', 'position_lng', 'available_bikes',
       'available_bike_stands', 'rounded_last_update',
       'rounded_last_update_5mins', 'temp', 'feels_like', 'humidity',
       'rain_1h', 'weather_desc', 'weather_brief', 'wind_speed', 'weatherid',
       'time_as_fraction', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2',
       'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6'],
      dtype='object')


In [79]:
station_number = x_set['number'].unique()
print(station_number)
x_set.columns

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  71  72  73  74
  75  76  77  78  79  80  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117]


Index(['number', 'time_as_fraction', 'day_of_week_0', 'day_of_week_1',
       'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
       'day_of_week_6', 'temp', 'feels_like'],
      dtype='object')

In [80]:
print(y_set)

        available_bike_stands
0                          30
1                          30
2                          30
3                          29
4                          29
...                       ...
622716                     40
622717                     40
622718                     40
622719                     40
622720                     40

[622721 rows x 1 columns]


In [81]:
maxv = y_set.max()

In [82]:
for num in station_number:
    x_set_num = x_set.loc[x_set['number'] == num]
    y_set_num = y_set[x_set['number'] == num]
    # Drop 'number' column correctly
    x_set_num = x_set_num.drop('number', axis=1)
    
    # One-hot encode correctly
    y_set_num = to_categorical(y_set_num, num_classes=41)  # Ensure num_classes matches your model's output
    
    # Convert to DataFrame if necessary (may not be needed if using TensorFlow/Keras directly)
    # y_set_num = pd.DataFrame(y_set_num)
    
    train_x = x_set_num.sample(frac=.8, replace=True)
    remainder_x = x_set_num.drop(train_x.index)
    y_set_num = pd.DataFrame(y_set_num)

    # Reset index if necessary to ensure alignment
    x_set_num.reset_index(drop=True, inplace=True)
    y_set_num.reset_index(drop=True, inplace=True)

    # Sample without replacement to avoid duplicate indices
    train_x = x_set_num.sample(frac=.8, replace=False)
    remainder_x = x_set_num.drop(train_x.index)

    # Ensure that y_set is correctly indexed and matched based on the new indexing
    y_train = y_set_num.iloc[train_x.index]
    remainder_y = y_set_num.iloc[remainder_x.index]
    print()
    # Define model as before...
    model = tf.keras.Sequential([
        tf.keras.layers.Normalization(input_shape=[10,], axis=None),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(41, activation='softmax', use_bias=True)
    ])
    
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy', 'mse', 'categorical_accuracy'])
    
    # Fit model using correctly shaped data
    model.fit(train_x, y_train, epochs=200, batch_size=50)
    
    # Evaluate model using correctly shaped data
    evaluation = model.evaluate(remainder_x, remainder_y, verbose=2)
    print(f"Model evaluation on remainder set: {evaluation}")



Epoch 1/200


c:\Users\thede\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 989us/step - accuracy: 0.0469 - categorical_accuracy: 0.0469 - loss: 8.4814 - mse: 0.0277      
Epoch 2/200
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step - accuracy: 0.0709 - categorical_accuracy: 0.0709 - loss: 3.5038 - mse: 0.0237
Epoch 3/200
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.0695 - categorical_accuracy: 0.0695 - loss: 3.4925 - mse: 0.0237      
Epoch 4/200
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.0773 - categorical_accuracy: 0.0773 - loss: 3.4637 - mse: 0.0236
Epoch 5/200
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - accuracy: 0.0939 - categorical_accuracy: 0.0939 - loss: 3.4311 - mse: 0.0235
Epoch 6/200
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.0869 - categorical_accuracy: 0.0869 - loss: 3.4190 - mse: 0.0235
Epoch 7/200
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - accuracy: 0.0898 - categorical_accuracy: 0.0898 - loss: 3.4156 - mse: 0.0235
Epoch 8/200
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.1052 - ca

c:\Users\thede\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 985us/step - accuracy: 0.0988 - categorical_accuracy: 0.0988 - loss: 7.7884 - mse: 0.0295       
Epoch 2/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.1964 - categorical_accuracy: 0.1964 - loss: 2.7661 - mse: 0.0224
Epoch 3/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - accuracy: 0.1799 - categorical_accuracy: 0.1799 - loss: 2.7850 - mse: 0.0226
Epoch 4/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - accuracy: 0.1618 - categorical_accuracy: 0.1618 - loss: 2.7931 - mse: 0.0227
Epoch 5/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step - accuracy: 0.1555 - categorical_accuracy: 0.1555 - loss: 2.7968 - mse: 0.0228
Epoch 6/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - accuracy: 0.1824 - categorical_accuracy: 0.1824 - loss: 2.7144 - mse: 0.0223
Epoch 7/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - accuracy: 0.1928 - categorical_accuracy: 0.1928 - loss: 2.7340 - mse: 0.0224
Epoch 8/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - accuracy: 0.1958 - categor

...

90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0599 - categorical_accuracy: 0.0599 - loss: 7.7342 - mse: 0.0276
Epoch 2/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.0582 - categorical_accuracy: 0.0582 - loss: 3.1446 - mse: 0.0235
Epoch 3/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - accuracy: 0.0640 - categorical_accuracy: 0.0640 - loss: 3.1767 - mse: 0.0236
Epoch 4/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.0658 - categorical_accuracy: 0.0658 - loss: 3.0756 - mse: 0.0233
Epoch 5/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.0706 - categorical_accuracy: 0.0706 - loss: 3.0506 - mse: 0.0233
Epoch 6/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step - accuracy: 0.0678 - categorical_accuracy: 0.0678 - loss: 3.0904 - mse: 0.0234
Epoch 7/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.0664 - categorical_accuracy: 0.0664 - loss: 3.0295 - mse: 0.0232
Epoch 8/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0642 - categorical_accura

c:\Users\thede\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 969us/step - accuracy: 0.0569 - categorical_accuracy: 0.0569 - loss: 6.7347 - mse: 0.0267
Epoch 2/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step - accuracy: 0.0524 - categorical_accuracy: 0.0524 - loss: 3.0884 - mse: 0.0233
Epoch 3/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - accuracy: 0.0823 - categorical_accuracy: 0.0823 - loss: 3.0407 - mse: 0.0232
Epoch 4/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.0825 - categorical_accuracy: 0.0825 - loss: 3.0631 - mse: 0.0233
Epoch 5/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.0720 - categorical_accuracy: 0.0720 - loss: 3.0288 - mse: 0.0232
Epoch 6/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - accuracy: 0.0811 - categorical_accuracy: 0.0811 - loss: 3.0189 - mse: 0.0232
Epoch 7/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - accuracy: 0.0798 - categorical_accuracy: 0.0798 - loss: 2.9908 - mse: 0.0231
Epoch 8/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - accuracy: 0.0738 - categorical_ac

c:\Users\thede\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 